In [1]:
%pip install -qU mem0ai azure-identity langchain langchain_voyageai pymongo

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

# 使用 Azure 时加上这行，否则会走到 Azure 的 tracing 系统，报401Error
os.environ["OPENAI_AGENTS_DISABLE_TRACING"] = "true"

# 导入 Azure OpenAI Python 客户端库
from openai import AzureOpenAI

# 初始化Azure OpenAI 客户端
# 将使用环境变量 OPENAI_API_KEY 中的 API 密钥
#openai_client = OpenAI()
openai_client = AzureOpenAI(
    api_version="2025-03-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment="gpt-4o",
)


In [3]:
from langchain_voyageai import VoyageAIEmbeddings


config = {
    "vector_store": {
        "provider": "mongodb",
        "config": {
            "db_name": "mem0_agent_memory_test",
            "collection_name": "extracted_memories",
            "mongo_uri": os.environ["MONGODB_URI"],
            "embedding_model_dims": 1024
        }
    },
    "embedder": {
        "provider": "langchain",
        "config": {
            "model": VoyageAIEmbeddings(
                model="voyage-4-large",
                voyage_api_key=os.getenv("VOYAGE_API_KEY")
            ),
            "embedding_dims": 1024
        }
    },
    "llm": {
        "provider": "azure_openai",
        "config": {
            "model": "gpt-4o",
            "azure_kwargs": {
                "api_key": os.getenv("OPENAI_API_KEY"),
                "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
                "azure_deployment": "gpt-4o",
                "api_version": "2025-03-01-preview"
            }
        }
    }
}

/Users/binzhou/Demo/agent_memory_demo/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 常量
CURRENT_USER_ID = "use001"

In [5]:
from mem0 import Memory
memory = Memory.from_config(config)

INFO:mem0.vector_stores.mongodb:Search index 'extracted_memories_vector_index' already exists in collection 'extracted_memories'.
INFO:mem0.vector_stores.mongodb:Search index 'mem0migrations_vector_index' already exists in collection 'mem0migrations'.
INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c0c3c749-98eb-4d92-9620-8283735e0a66'.


In [6]:
messages = [
    {"role": "user", "content": "Hi, I'm Alex. I love basketball and gaming."},
    {"role": "assistant", "content": "Hey Alex! I'll remember your interests."}
]
memory.add(messages, user_id=CURRENT_USER_ID)

# m.search(query="What does Alex like?", user_id=CURRENT_USER_ID)
# m.delete(user_id=CURRENT_USER_ID)

INFO:httpx:HTTP Request: POST https://newenglandhackopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/voyageai/voyage-4-large/resolve/main/tokenizer.json "HTTP/1.1 307 Temporary Redirect"
INFO:httpx:HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/voyageai/voyage-4-large/bb931c2635a93efe400c24741363d8ff61d7bb32/tokenizer.json "HTTP/1.1 200 OK"
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 3 documents.
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 3 documents.
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 3 documents.
INFO:mem0.memory.main:Total existing memories: 3
INFO:httpx:HTTP Request: POST https://newenglandhackopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-03-01-preview "HTTP/1.1 200 OK"
INFO:mem0.memory.main:{'id': '0', 'text': 'Name is Alex', 'event': 'NONE

{'results': []}

In [8]:
results = memory.search("What do you know about me?", user_id=CURRENT_USER_ID)
print(results)

INFO:mem0.vector_stores.mongodb:Retrieved document with ID 'c0c3c749-98eb-4d92-9620-8283735e0a66'.
INFO:mem0.vector_stores.mongodb:Vector search completed. Found 3 documents.


{'results': [{'id': 'f50f8734-083e-42a1-9183-74e50270d76d', 'memory': 'Loves gaming', 'hash': '8b423caec418a7703c88e0301440ab77', 'metadata': None, 'score': 0.6230825185775757, 'created_at': '2026-02-03T21:53:50.424718-08:00', 'updated_at': None, 'user_id': 'use001'}, {'id': 'abb6c64b-7e12-4536-8572-ccfb8632fa96', 'memory': 'Loves basketball', 'hash': 'af2e8a68f80d5ece763865e3a5d5ce15', 'metadata': None, 'score': 0.6222937107086182, 'created_at': '2026-02-03T21:53:50.226080-08:00', 'updated_at': None, 'user_id': 'use001'}, {'id': '6ee22320-d1d1-417d-a120-eaaadcedc2c6', 'memory': 'Name is Alex', 'hash': 'd0fccc8fa47f7a149ee95750c37bb0ca', 'metadata': None, 'score': 0.6137574911117554, 'created_at': '2026-02-03T21:53:50.026598-08:00', 'updated_at': None, 'user_id': 'use001'}]}
